# Excercise 1B.1

## Imports, constants
Import numpy, set constant light speed.

In [96]:
import numpy as np

LIGHT_SPEED = 299792.458

## Calculate Jacobian
This function calculates the Jacobian of system (2). 

It needs the ```location```, where you estimated the person could be, and the estimated time difference. This is needed in a vector s.t. 
$\begin{bmatrix} x \\ y \\ z \\ D \end{bmatrix}$.

The ```positions``` of the satellites, in a ```numpy``` array (matrix) s.t.:
$$
\begin{bmatrix} 
a_1 & b_1 & c_1 \\
a_2 & b_2 & c_2 \\
a_3 & b_3 & c_3 \\
a_4 & b_4 & c_4 \\
\end{bmatrix}
$$

The ```transmission_times```, between the receiver and the satellites in a ```numpy``` array s.t.:
$\begin{bmatrix} t_1 \\ t_2 \\ t_3 \\ t_4 \end{bmatrix}$

In [97]:
def calculate_jacobian(
    location: np.ndarray, 
    positions: np.ndarray, 
    transmission_times: np.ndarray
):
    jacobian = np.zeros((4,4))
    
    location_x = location[0,0]
    location_y = location[1,0]
    location_z = location[2,0]
    time_difference = location[3,0]

    jacobian[:,0] = 2 * (location_x - positions[:, 0])
    jacobian[:,1] = 2 * (location_y - positions[:, 1])
    jacobian[:,2] = 2 * (location_z - positions[:, 2])
    jacobian[:,3] = 2 * (LIGHT_SPEED**2) * transmission_times - 2 * (LIGHT_SPEED**2) * time_difference
 
    return jacobian

## Calculate System
This function calculates the values of the vector below.
$$
\begin{bmatrix}
(x - a_1)^2 + (y - b_1)^2 + (z - c_1)^2 - (C(t_1 - D))^2 \\ 
(x - a_2)^2 + (y - b_2)^2 + (z - c_2)^2 - (C(t_2 - D))^2 \\ 
(x - a_3)^2 + (y - b_3)^2 + (z - c_3)^2 - (C(t_3 - D))^2 \\ 
(x - a_4)^2 + (y - b_4)^2 + (z - c_4)^2 - (C(t_4 - D))^2
\end{bmatrix}
$$

It needs the ```location```, where you estimated the person could be, and the estimated time difference. This is needed in a ```numpy``` array (vector) s.t. 
$\begin{bmatrix} x \\ y \\ z \\ D \end{bmatrix}$.

The ```positions``` of the satellites, in a ```numpy``` array (matrix) s.t.:
$$
\begin{bmatrix} 
a_1 & b_1 & c_1 \\
a_2 & b_2 & c_2 \\
a_3 & b_3 & c_3 \\
a_4 & b_4 & c_4 \\
\end{bmatrix}
$$

The ```transmission_times```, between the receiver and the satellites in a ```numpy``` array s.t.:
$\begin{bmatrix} t_1 \\ t_2 \\ t_3 \\ t_4 \end{bmatrix}$

In [98]:
def functions(
    location: np.ndarray, 
    positions: np.ndarray, 
    transmission_times: np.ndarray
):
    estimated_location_x = location[0,0]
    estimated_location_y = location[1,0]
    estimated_location_z = location[2,0]

    time_difference = location[3,0]
    result = np.zeros((4, 1))

    result[:, 0] = (estimated_location_x - positions[:, 0])**2 + (estimated_location_y - positions[:, 1])**2 + (estimated_location_z - positions[:, 2])**2 - (LIGHT_SPEED * (transmission_times - time_difference))**2
    return result

## Estimate location
This function uses Newtons Multivariate Method, to estimate the location using the known locations and transmission time. 

The ```initial_estimation``` as a ```numpy``` array (vector) is needed.

The ```positions``` of the satellites, in a ```numpy``` array (matrix) s.t.:
$$
\begin{bmatrix} 
a_1 & b_1 & c_1 \\
a_2 & b_2 & c_2 \\
a_3 & b_3 & c_3 \\
a_4 & b_4 & c_4 \\
\end{bmatrix}
$$

The ```transmission_times```, between the receiver and the satellites in a ```numpy``` array s.t.:
$\begin{bmatrix} t_1 \\ t_2 \\ t_3 \\ t_4 \end{bmatrix}$

Also the maximum ```iterations``` are needed, as an ```int```.

In [99]:
def estimate_location(
    inital_estimation: np.ndarray,
    positions: np.ndarray, 
    transmission_times: np.ndarray,
    iterations: int,
    minimum_step_size: float
):
    estimation: np.ndarray = inital_estimation
    
    for _ in range(iterations):
        result = functions(estimation, positions, transmission_times)
        jacobian = calculate_jacobian(estimation, positions, transmission_times)
        H = np.linalg.solve(jacobian, result)
        estimation = estimation - H

        # Break if step_size is within minimum_step_size
        if np.linalg.norm(H) < minimum_step_size:
            break

        # another break needed, but don't understand
    
    return estimation

## Main
This runs our program using the values given.

In [100]:
def main():
    locations = np.array([
        [15600, 7540, 20140], 
        [18760, 2750, 18610], 
        [17610, 14630, 13480], 
        [19170, 610, 18390]
    ])

    transmission_times = np.array([
        0.07074,
        0.07220,
        0.07690,
        0.07242
    ])

    initial_estimation = np.array([[0], [0], [6370], [0]])
    max_iterations = 100
    minimum_step_size = 0.0000000001 # 1 micrometer
    x, y, z, time_difference = estimate_location(initial_estimation, locations, transmission_times, max_iterations, minimum_step_size).tolist()

    print(f"{x=}, {y=}, {z=}, {time_difference=}")

main()

x=[-41.77270957082428], y=[-16.789194106523492], z=[6370.059559223344], time_difference=[-0.0032015658295941082]
